# Задание 6. Классификация новостей

### Данные
Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

## Задание 6.1 

Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):

- pymorphy2
- русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
- [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
## Задание 6.2

Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

## Задание 6.3

Реализовать алгоритм классификации документа по категориям, посчитать точноcть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия
    

## Задание 6.4* 

Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

#### Обработка данных и получение токенов

In [1]:
import pandas as pd
import regex as re
import numpy as np
import warnings
import nltk

from sklearn import metrics
from sklearn import naive_bayes
from gensim.models import Word2Vec
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
warnings.filterwarnings("ignore")

In [3]:
train_data = pd.read_csv("news/news_train.txt", sep="\t", header=None)
test_data = pd.read_csv("news/news_test.txt", sep="\t", header=None)
train_data.rename(columns={0: 'topic', 1: 'header', 2: 'text'}, inplace=True)
test_data.rename(columns={0: 'topic', 1: 'header', 2: 'text'}, inplace=True)

In [4]:
train_data

,topic,header,text
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...
...,...,...,...
14995,life,Составлен рейтинг лучших европейских пляжей 20...,Опубликован рейтинг лучших европейских пляжей ...
14996,media,В «Снобе» объяснили причину смены формата,Генеральный директор «Сноб медиа» Марина Гевор...
14997,economics,Минфин предложил штрафовать за биткоины на 50 ...,"Минфин разработал законопроект, устанавливающи..."
14998,life,Мэл Гибсон заплатит бывшей подруге 750 тысяч д...,Актер и режиссер Мэл Гибсон выплатит своей быв...


In [5]:
sb_stemmer = SnowballStemmer("russian")

In [6]:
def prepare_data(text):
    new_text = text.lower()
    new_text = re.sub(r'[\d\.[\]«»,"\'%-?:—!;\(\)*]', ' ', new_text)
    new_text = re.sub(r'\s{2,}', ' ', new_text)
    tokens = [t for t in new_text.split()]

    words=[]
    for t in tokens:
        if len(t)>1:
            words.append(sb_stemmer.stem(t))
    
    res = " ".join(words).strip()
    return res

In [7]:
cleaned_header_train = []
cleaned_text_train = []

for header in train_data['header']:
    cleaned_header_train.append(prepare_data(header))
    
for text in train_data['text']:
    cleaned_text_train.append(prepare_data(text))

In [8]:
print(train_data['text'][0])

Нападающий «Вашингтон Кэпиталз» Александр Овечкин передал детской хоккейной школе автомобиль, полученный им после окончания Матча всех звезд Национальной хоккейной лиги (НХЛ). Об этом сообщается на официальном сайте лиги.Автомобиль Honda Accord был подарен хоккеисту по решению спонсоров мероприятия. Игрок НХЛ пожертвовал машину спортивной школе Nova Cool Cats Special Hockey Inc., которая расположена в штате Вирджиния.Овечкин общается с 10-летней девочкой Анной Шоб с синдромом Дауна, которая занимается в этой школе и является поклонницей спортсмена. В сентябре форвард пообедал вместе с юной хоккеисткой в японском ресторане.Матч всех звезд НХЛ в Коламбусе (штат Огайо) завершился победой команды «Джонатана Тэйвза» над командой «Ника Фолиньо» со счетом 17:12. Овечкин выступал за проигравший коллектив. Россиянин отметился тремя результативными передачами.


In [9]:
print(cleaned_text_train[0], "\n")

напада вашингтон кэпиталз александр овечкин переда детск хоккейн школ автомобил получен им посл окончан матч всех звезд национальн хоккейн лиг нхл об эт сообща на официальн сайт лиг автомобил honda accord был подар хоккеист по решен спонсор мероприят игрок нхл пожертвова машин спортивн школ nova cool cats special hockey inc котор располож штат вирджин овечкин обща летн девочк ан шоб синдром даун котор занима эт школ явля поклонниц спортсм сентябр форвард пообеда вмест юн хоккеистк японск ресторан матч всех звезд нхл коламбус штат огай заверш побед команд джоната тэйвз над команд ник фолин со счет овечкин выступа за проигра коллект россиянин отмет трем результативн передач 



In [10]:
topics = train_data.topic.values
headers = cleaned_header_train
texts = cleaned_text_train
arr = []
for i in range(len(topics)):
    arr.append([topics[i], headers[i], texts[i]])
df = pd.DataFrame(arr, columns=['topic', 'header', 'text'])
df

,topic,header,text
0,sport,овечкин пожертвова детск хоккейн школ автомобил,напада вашингтон кэпиталз александр овечкин пе...
1,culture,рекордн дорог стат май призна подделк,власт мексик объяв подделк стат май прода на э...
2,science,samsung представ флагма защищен корпус,южнокорейск samsung анонсирова защищен верс св...
3,sport,футболист спартак снял четырехматчев дисквалиф...,контрольн дисциплинарн комитет кдк рфс снял ди...
4,media,hopes fears объедин the village,интернет издан hopes fears объяв сво слиян сай...
...,...,...,...
14995,life,составл рейтинг лучш европейск пляж год,опубликова рейтинг лучш европейск пляж год топ...
14996,media,сноб объясн причин смен формат,генеральн директор сноб мед марин геворкя объя...
14997,economics,минфин предлож штрафова за биткоин на тысяч рубл,минфин разработа законопроект устанавлива штра...
14998,life,мэл гибсон заплат бывш подруг тысяч доллар,актер режиссер мэл гибсон выплат сво бывш подр...


#### Обучение Word Embedding (Word2Vec) на тренировочных данных

In [11]:
model = Word2Vec()
headers = df.header.apply(lambda x: x.split(" ")).values
texts = df.text.apply(lambda x: x.split(" ")).values
model.build_vocab(headers + texts)

In [12]:
model.train(headers + texts, total_examples=len(texts), epochs=30)

(71001124, 79712490)

In [13]:
model.wv.key_to_index

{'на': 0,
 'по': 1,
 'год': 2,
 'что': 3,
 'эт': 4,
 'не': 5,
 'котор': 6,
 'из': 7,
 'был': 8,
 'он': 9,
 'за': 10,
 'как': 11,
 'для': 12,
 'росс': 13,
 'сообща': 14,
 'об': 15,
 'компан': 16,
 'такж': 17,
 'сво': 18,
 'от': 19,
 'ег': 20,
 'до': 21,
 'так': 22,
 'российск': 23,
 'нов': 24,
 'процент': 25,
 'доллар': 26,
 'будет': 27,
 'перв': 28,
 'посл': 29,
 'миллион': 30,
 'врем': 31,
 'при': 32,
 'сам': 33,
 'стал': 34,
 'рубл': 35,
 'со': 36,
 'заяв': 37,
 'тысяч': 38,
 'тем': 39,
 'слов': 40,
 'друг': 41,
 'одн': 42,
 'дан': 43,
 'е': 44,
 'миллиард': 45,
 'сайт': 46,
 'стран': 47,
 'тог': 48,
 'во': 49,
 'получ': 50,
 'том': 51,
 'игр': 52,
 'сша': 53,
 'лет': 54,
 'ран': 55,
 'работ': 56,
 'однак': 57,
 'the': 58,
 'бол': 59,
 'явля': 60,
 'но': 61,
 'их': 62,
 'фильм': 63,
 'то': 64,
 'проект': 65,
 'материал': 66,
 'мест': 67,
 'все': 68,
 'больш': 69,
 'сообщ': 70,
 'мир': 71,
 'уж': 72,
 'ил': 73,
 'нача': 74,
 'суд': 75,
 'под': 76,
 'американск': 77,
 'отмет': 78,
 'гр

In [14]:
model.wv.most_similar('миллион')

[('тысяч', 0.8625084161758423),
 ('миллиард', 0.831612765789032),
 ('полумиллион', 0.7178180813789368),
 ('полумиллиард', 0.7004258632659912),
 ('триллион', 0.6949099898338318),
 ('полмиллиард', 0.6665676236152649),
 ('полмиллион', 0.6212290525436401),
 ('млн', 0.6073290109634399),
 ('цент', 0.5700018405914307),
 ('процент', 0.506363570690155)]

In [15]:
model.wv.most_similar('женщин')

[('мужчин', 0.7788180708885193),
 ('девушк', 0.6836047172546387),
 ('подростк', 0.6512607932090759),
 ('девушек', 0.647753894329071),
 ('люд', 0.6430768370628357),
 ('девочк', 0.6178980469703674),
 ('девочек', 0.6078830361366272),
 ('родител', 0.6014395952224731),
 ('дет', 0.5926093459129333),
 ('мальчик', 0.584985077381134)]

#### Реализация классификации документов по категориям

In [17]:
lr_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr_estimator', LogisticRegression(random_state=42))])
lr_clf.fit(train_data.text, train_data.topic)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr_estimator', LogisticRegression(random_state=42))])

In [18]:
predicted_lr = lr_clf.predict(test_data.text)
print(metrics.classification_report(predicted_lr, test_data.topic))

              precision    recall  f1-score   support

    business       0.23      0.78      0.36        27
     culture       0.92      0.92      0.92       427
   economics       0.91      0.82      0.87       473
      forces       0.88      0.83      0.85       261
        life       0.90      0.81      0.86       461
       media       0.88      0.84      0.86       422
     science       0.86      0.89      0.88       451
       sport       0.97      0.97      0.97       421
       style       0.71      1.00      0.83        37
      travel       0.37      1.00      0.54        20

    accuracy                           0.87      3000
   macro avg       0.76      0.89      0.79      3000
weighted avg       0.89      0.87      0.88      3000



In [20]:
params_lr = {'lr_estimator__C': [20.0, 10.0, 5.0, 1.0, 0.5, 0.1]}
grid_lr = GridSearchCV(estimator=lr_clf, param_grid=params_lr)
grid_lr.fit(train_data.text, train_data.topic)
print("Лучшее значение параметра: ", grid_lr.best_params_)
print("Лучший результат модели: ", grid_lr.best_score_)

Лучшее значение параметра:  {'lr_estimator__C': 20.0}
Лучший результат модели:  0.8718666666666668


In [21]:
predicted_grid_lr = grid_lr.predict(test_data.text)
print(metrics.classification_report(predicted_grid_lr, test_data.topic))

              precision    recall  f1-score   support

    business       0.49      0.77      0.60        57
     culture       0.93      0.92      0.93       430
   economics       0.91      0.87      0.89       448
      forces       0.89      0.84      0.86       262
        life       0.91      0.85      0.88       445
       media       0.88      0.87      0.87       407
     science       0.86      0.90      0.88       448
       sport       0.97      0.98      0.98       420
       style       0.83      0.96      0.89        45
      travel       0.65      0.92      0.76        38

    accuracy                           0.89      3000
   macro avg       0.83      0.89      0.85      3000
weighted avg       0.90      0.89      0.89      3000



In [22]:
nb_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb_estimator', naive_bayes.MultinomialNB())])
nb_clf.fit(train_data.text, train_data.topic)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('nb_estimator', MultinomialNB())])

In [23]:
predicted_nb = nb_clf.predict(test_data.text)
print(metrics.classification_report(predicted_nb, test_data.topic))

              precision    recall  f1-score   support

    business       0.00      0.00      0.00         0
     culture       0.92      0.87      0.89       450
   economics       0.97      0.66      0.78       629
      forces       0.51      0.93      0.65       134
        life       0.77      0.87      0.82       368
       media       0.87      0.73      0.79       480
     science       0.87      0.81      0.84       503
       sport       0.99      0.96      0.97       436
       style       0.00      0.00      0.00         0
      travel       0.00      0.00      0.00         0

    accuracy                           0.81      3000
   macro avg       0.59      0.58      0.57      3000
weighted avg       0.89      0.81      0.84      3000



In [24]:
params_nb = {'nb_estimator__alpha': [0, 0.5, 1]}
grid_nb = GridSearchCV(nb_clf, params_nb)
grid_nb.fit(train_data.text, train_data.topic)
print(grid_nb.best_params_, grid_nb.best_score_)

{'nb_estimator__alpha': 0} 0.8132666666666666


In [25]:
predicted_grid_nb = grid_nb.predict(test_data.text)
print(metrics.classification_report(predicted_grid_nb, test_data.topic))

              precision    recall  f1-score   support

    business       0.21      0.61      0.31        31
     culture       0.89      0.85      0.87       445
   economics       0.89      0.82      0.85       464
      forces       0.76      0.81      0.78       232
        life       0.83      0.79      0.81       438
       media       0.81      0.72      0.76       454
     science       0.83      0.80      0.82       480
       sport       0.94      0.99      0.96       402
       style       0.56      0.91      0.69        32
      travel       0.41      1.00      0.58        22

    accuracy                           0.82      3000
   macro avg       0.71      0.83      0.74      3000
weighted avg       0.84      0.82      0.83      3000

